# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("output_data/cities.csv")
cities

,City,Country,Date,Lat,Lng,Max Temp,Wind Speed,Cloudiness,Humidity
0,Kahului,US,1592408814,20.89,-156.47,75.20,12.75,20,73
1,Mount Isa,AU,1592408814,-20.73,139.50,68.00,9.17,0,42
2,Roald,NO,1592408815,62.58,6.13,71.01,8.05,49,72
3,El Alto,PE,1592408815,-4.27,-81.21,73.40,25.28,22,69
4,Saskylakh,RU,1592408815,71.92,114.08,53.73,9.15,0,63
...,...,...,...,...,...,...,...,...,...
551,Utiel,ES,1592408970,39.57,-1.20,82.00,13.51,47,22
552,Tomohon,ID,1592408971,1.33,124.84,77.00,2.24,20,94
553,Jambi City,ID,1592408971,-1.60,103.62,75.63,2.06,89,91
554,Puerto Escondido,MX,1592408971,15.85,-97.07,86.00,6.69,40,70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
LatLong = cities[['Lat','Lng']]
gmaps.configure(api_key=g_key)
# m=gmaps.Map()
# m.add_layer(gmaps.Heatmap(data=LatLong))

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(LatLong, weights=cities['Humidity'],max_intensity=100, point_radius=5.0)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
ideal_weather = cities.loc[(cities['Cloudiness']<=10) & (cities['Humidity']<=20) & (cities['Max Temp'] >90)]
ideal_weather

,City,Country,Date,Lat,Lng,Max Temp,Wind Speed,Cloudiness,Humidity
81,Darya Khan,PK,1592408837,31.79,71.10,104.81,5.61,0,7
100,Arlit,NE,1592408841,18.74,7.39,106.48,18.12,0,7
165,Assiut,EG,1592408860,27.18,31.18,107.60,16.11,0,13
229,Adrar,MR,1592408879,20.50,-10.07,105.87,25.12,0,12
232,Atar,MR,1592408742,20.52,-13.05,107.40,25.86,0,11
325,Araouane,ML,1592408907,18.90,-3.53,114.06,8.99,0,5
351,Taoudenni,ML,1592408914,22.68,-3.98,112.96,18.19,0,6
397,Nālūt,LY,1592408926,30.33,10.85,96.62,2.75,0,10
543,Riyadh,SA,1592408823,24.69,46.72,107.60,13.87,10,8
550,Waddān,LY,1592408970,29.16,16.14,93.94,12.91,0,11


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = pd.DataFrame(ideal_weather).reset_index(drop=True)
hotel_df

,City,Country,Date,Lat,Lng,Max Temp,Wind Speed,Cloudiness,Humidity
0,Darya Khan,PK,1592408837,31.79,71.10,104.81,5.61,0,7
1,Arlit,NE,1592408841,18.74,7.39,106.48,18.12,0,7
2,Assiut,EG,1592408860,27.18,31.18,107.60,16.11,0,13
3,Adrar,MR,1592408879,20.50,-10.07,105.87,25.12,0,12
4,Atar,MR,1592408742,20.52,-13.05,107.40,25.86,0,11
5,Araouane,ML,1592408907,18.90,-3.53,114.06,8.99,0,5
6,Taoudenni,ML,1592408914,22.68,-3.98,112.96,18.19,0,6
7,Nālūt,LY,1592408926,30.33,10.85,96.62,2.75,0,10
8,Riyadh,SA,1592408823,24.69,46.72,107.60,13.87,10,8
9,Waddān,LY,1592408970,29.16,16.14,93.94,12.91,0,11


In [32]:
# set up a parameters dictionary
params = {
    "query": "hotel",
    "location": "",
    "radius":5000,
    "key": g_key
}

# set a list to keep hotel results in
hotels = []

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# loop through hotel_df locations to find hotels
for index, row in hotel_df.iterrows():
#     lat = hotel_df['Lat']
#     lng = hotel_df['Lng']

    params['location']=f"{hotel_df['Lat']}, {hotel_df['Lng']}"
    response = requests.get(base_url, params=params).json()
    results = response['results']
    print(results)
#     try:
#         print(f"Closest hotel is {results[1]['name']}.")
#         hotel.append(results[1]['name'])
#     except (KeyError, IndexError):
#         print("Error... Skipping")

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'narrowed_city_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display Map